In [1]:
# import external libraries

import math
import random
import numbers

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

#pytorch utility imports

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid



from PIL import Image, ImageOps, ImageEnhance

In [2]:
train_data = pd.read_csv('C:/Users/rkvja/Downloads/MNIST/train.csv')
test_data = pd.read_csv('C:/Users/rkvja/Downloads/MNIST/test.csv')

print('Train size: ', train_data.shape)
print('Test size: ', test_data.shape)

train_data.columns
test_data.columns

Train size:  (42000, 785)
Test size:  (28000, 784)


Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [3]:
batch_size = 200
num_epochs= 5
num_classes= 10

In [5]:
class MNISTDataset(Dataset):
    """MNIST-data-set"""
    
    def __init__(self, file_path, 
                 transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), 
                     transforms.Normalize(mean=(0.5,), std=(0.5,))])
                ):
        
        df = pd.read_csv(file_path)
        
        if len(df.columns) == 784:
            # test data
            self.X = df.values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        else:
            # training data
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:,0].values)
            
        self.transform = transform
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])
        
train_dataset = MNISTDataset('C:/Users/rkvja/Downloads/MNIST/train.csv', transform= transforms.Compose(
                            [transforms.ToPILImage(),transforms.ToTensor(), 
                             transforms.Normalize(mean=(0.5,), std=(0.5,))]))
test_dataset = MNISTDataset('C:/Users/rkvja/Downloads/MNIST/test.csv')

In [6]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
dataiteration = iter(train_loader)
images, labels = dataiteration.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([200, 1, 28, 28])
torch.Size([200])


In [8]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
          
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(64 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
          
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x

In [9]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [10]:
def train(epoch):
    model.train()
    exp_lr_scheduler.step()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.data))

In [11]:

def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).data

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [12]:
for epoch in range(num_epochs):
    train(epoch)
    evaluate(train_loader)

C:\Users\rkvja\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train Epoch: 0 [20000/42000 (48%)]	Loss: 0.114908
Train Epoch: 0 [40000/42000 (95%)]	Loss: 0.091957


<ipython-input-11-de41f3bac786>:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
C:\Users\rkvja\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Average loss: 0.0529, Accuracy: 41373/42000 (98.507%)

Train Epoch: 1 [20000/42000 (48%)]	Loss: 0.076638
Train Epoch: 1 [40000/42000 (95%)]	Loss: 0.086818

Average loss: 0.0304, Accuracy: 41619/42000 (99.093%)

Train Epoch: 2 [20000/42000 (48%)]	Loss: 0.025823
Train Epoch: 2 [40000/42000 (95%)]	Loss: 0.069471

Average loss: 0.0235, Accuracy: 41705/42000 (99.298%)

Train Epoch: 3 [20000/42000 (48%)]	Loss: 0.054988
Train Epoch: 3 [40000/42000 (95%)]	Loss: 0.049243

Average loss: 0.0198, Accuracy: 41762/42000 (99.433%)

Train Epoch: 4 [20000/42000 (48%)]	Loss: 0.065226
Train Epoch: 4 [40000/42000 (95%)]	Loss: 0.030476

Average loss: 0.0168, Accuracy: 41805/42000 (99.536%)



In [13]:
def prediciton(data_loader):
    model.eval()
    test_set_pred = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        data = Variable(data, volatile=True)
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_set_pred = torch.cat((test_set_pred, pred), dim=0)
        
    return test_set_pred

In [14]:
test_set_pred = prediciton(test_loader)


<ipython-input-13-7b9ac7217c46>:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)


In [17]:
test_set_pred

tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]])

In [19]:
df_export = pd.DataFrame(np.c_[np.arange(1, len(test_dataset)+1)[:,None], test_set_pred.numpy()], 
                      columns=['Id of Image', 'Label of Image'])

df_export.head()

,Id of Image,Label of Image
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [20]:
df_export.to_csv('my_submission.csv', index=False)
